In [1]:
# A small price to pay to have notebook directory! 
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from src.graph import create_graph, create_graph_by_symbol, draw_labeled_multigraph
import os
from ast import literal_eval
import subprocess
from collections import Counter
import itertools as it

In [6]:
df = pd.read_csv('/workspaces/uniswap-v3-crawler/data/snapshots/snapshot_22358551.csv.gz')
df = df.sort_values('liquidity')
df['other'] = df['other'].apply(literal_eval)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/uniswap-v3-crawler/data/snapshots/snapshot_22358551.csv.gz'

## Prepare the graph

### Liquidity 0
First we remove pools with liquidity 0.

Copilot:
 * A pool with zero liquidity indicates that there are no assets available for trading in that pool.
 * To have liquidity of 0, all assets must be removed from the pool or it must be initialized with no assets.

In [13]:
G = create_graph(df, apply_fee=False, remove_low_degree_nodes=True)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 2184
Number of edges: 10577


### Stats

In [14]:
print(f"Number of nodes after removal: {G.number_of_nodes()}")
print(f"Number of edges after removal: {G.number_of_edges()}")

# Count the frequency of the 'version' property of edges
version_counts = Counter(data['version'] for _, _, data in G.edges(data=True))

print("\nPool versions:")
# Print the frequency of each version
for version, count in version_counts.items():
    print(f"Pool version: {version}, Count: {count}")

# Calculate the degree of each node
node_degrees = dict(G.degree())

# Sort nodes by degree in descending order
sorted_nodes = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)

# Extract the top 5 nodes and their degrees
top_5_tokens = sorted_nodes[:5]

print("\nToken domination:")
# Print the top 5 token symbols and their degrees
for node, degree in top_5_tokens:
    token_symbol = df[df['token0_id'] == node]['token0_symbol'].iloc[0] if node in df['token0_id'].values else \
                    df[df['token1_id'] == node]['token1_symbol'].iloc[0] if node in df['token1_id'].values else 'Unknown'
    print(f"Token: {token_symbol}, Degree: {degree}")

Number of nodes after removal: 2184
Number of edges after removal: 10577

Pool versions:
Pool version: v2, Count: 2185
Pool version: v3, Count: 5890
Pool version: v4, Count: 2502

Token domination:
Token: WETH, Degree: 5182
Token: ETH, Degree: 1301
Token: USDC, Degree: 1281
Token: USDT, Degree: 941
Token: WBTC, Degree: 290


In [15]:
nx.negative_edge_cycle(G, weight='weight')

True

In [16]:
num_connected_components = nx.number_weakly_connected_components(G)
print(f"Number of weakly connected components: {num_connected_components}")

Number of weakly connected components: 95


In [8]:
# Calculate the treewidth of the graph
treewidth, decomposition = nx.approximation.treewidth_min_degree(nx.Graph(G.to_undirected()))
print(f"Treewidth of the graph: {treewidth}")

Treewidth of the graph: 14


## Find artbitrage Triangles